In [224]:
import dash
from dash import dcc, html, Input, Output, State, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
import numpy as np
from jupyter_dash import JupyterDash
import pdfkit
import base64
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pyodbc
from urllib.parse import quote_plus
import io
import oracledb
from datetime import datetime
from dash.exceptions import PreventUpdate
from concurrent.futures import ThreadPoolExecutor
import os

In [225]:
# Parámetros de conexión
server = '10.0.0.133'
database = 'DB_GENERAL'
database2= 'DB_TRAMITE_DOCUMENTARIO'  # Reemplaza con el nombre de tu base de datos
username = 'sa'
password = 'Essalud23**'
def create_connection():
    conn = None
    try:
        # Establecer conexión
        conn = pyodbc.connect(r'DRIVER={ODBC Driver 18 for SQL Server};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={username};'
        f'PWD={password};'
        'TrustServerCertificate=yes;')
        print("Conexión exitosa.")
    except Exception as e:
        print(f"Error al conectar: {e}")
    return conn

In [226]:
var="GG"

In [227]:
def fetch_data(var):
    conn = create_connection()
    if conn is None:
        raise ConnectionError("No se pudo establecer la conexión a la base de datos.")
    try:
        query = f"""
SELECT 
    d.hoja_tramite,
    d.tipo_hoja,
    cdi.DESCRIPCION,
    d.INDICATIVO_OFICIO,
    p.RAZON_SOCIAL,
    d.FECHA_RECEPCION,
    d.ASUNTO,
    md.AUDIT_REC,
    do.SIGLAS AS SIGLAS_ORIGEN,
    do.DEPENDENCIA AS DEPENDENCIA_ORIGEN,
    dd.SIGLAS AS SIGLAS_DESTINO,
    dd.DEPENDENCIA AS DEPENDENCIA_DESTINO,
    ht.APELLIDOS_TRABAJADOR,
    ht.NOMBRES_TRABAJADOR,
    ed.ID_ESTADO_DOCUMENTO,
    ed.DESCRIPCION AS ESTADO_DOCUMENTO,
    md.AUDIT_TRAB_DER AS FECHA_DELEGADO,
    md.derivado,
    md.finalizado,

    -- Columna DIFERENCIA_DIAS
    DATEDIFF(DAY, md.AUDIT_TRAB_DER, GETDATE()) AS DIFERENCIA_DIAS,

    -- Columna ESTADO
    CASE
        WHEN ed.ID_ESTADO_DOCUMENTO <> 2 
          AND ed.ID_ESTADO_DOCUMENTO <> 10 
          AND md.derivado = 0 
          AND md.finalizado = 0 THEN 'Pendiente'
        ELSE 'Otros'
    END AS ESTADO

FROM 
    DB_TRAMITE_DOCUMENTARIO.web_tramite.MOVIMIENTO_DOCUMENTO md
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO d ON md.ID_DOCUMENTO = d.ID_DOCUMENTO
LEFT OUTER JOIN 
    DB_GENERAL.dbo.PERSONA p ON d.ID_PERSONA = p.ID
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.dbo.CLASE_DOCUMENTO_INTERNO cdi ON d.ID_CLASE_DOCUMENTO_INTERNO = cdi.ID_CLASE_DOCUMENTO_INTERNO
LEFT OUTER JOIN 
    DB_TRAMITE_DOCUMENTARIO.dbo.ESTADO_DOCUMENTO ed ON ed.ID_ESTADO_DOCUMENTO = d.ID_ESTADO_DOCUMENTO
LEFT OUTER JOIN 
    DB_GENERAL.jcardenas.H_DEPENDENCIA do ON do.CODIGO_DEPENDENCIA = md.ID_DEPENDENCIA_ORIGEN 
LEFT OUTER JOIN 
    DB_GENERAL.jcardenas.H_DEPENDENCIA dd ON dd.CODIGO_DEPENDENCIA = md.ID_DEPENDENCIA_DESTINO
LEFT JOIN 
    DB_GENERAL.jcardenas.H_TRABAJADOR ht ON ht.CODIGO_TRABAJADOR = md.codigo_trabajador
WHERE
    (do.SIGLAS LIKE '{var}' OR dd.SIGLAS LIKE '{var}')
    AND
    md.AUDIT_TRAB_DER = (
        SELECT MAX(sub_md.AUDIT_TRAB_DER)
        FROM DB_TRAMITE_DOCUMENTARIO.web_tramite.MOVIMIENTO_DOCUMENTO sub_md
        LEFT OUTER JOIN DB_TRAMITE_DOCUMENTARIO.web_tramite.DOCUMENTO sub_d ON sub_md.ID_DOCUMENTO = sub_d.ID_DOCUMENTO
        WHERE sub_d.hoja_tramite = d.hoja_tramite
    )
        """
        result = pd.read_sql(query, conn)
        result["Hoja de trámite"]=result["hoja_tramite"]+"-"+result["tipo_hoja"]
        result = result[result["ESTADO"] == "Pendiente"]
        result["Trabajador"]= result["NOMBRES_TRABAJADOR"]+ ", "+result["APELLIDOS_TRABAJADOR"]
        result[['parte_izquierda', 'Destino']] = result['SIGLAS_DESTINO'].str.rsplit('-', n=1, expand=True)
        result['Destino_vf'] = np.where(
        result['Destino'].isna(),  # Si 'Destino' está vacío (equivalente a BLANK() en DAX)
        result['SIGLAS_DESTINO'] + " - " + result['DEPENDENCIA_DESTINO'],  # Si está vacío
        result['Destino'] + " - " + result['DEPENDENCIA_DESTINO'])  # Si no está vacío
        result['tip_hoja'] = result['tipo_hoja'].apply(lambda x: 'Interno' if x == 'I' else ('Externo' if x == 'E' else None))
        result['FECHA_DELEGADO'] = pd.to_datetime(result['FECHA_DELEGADO'])
        result['FECHA_DELEGADO'] = result['FECHA_DELEGADO'].dt.strftime('%Y-%m-%d %H:%M:%S')
    finally:
        conn.close()  # Cerrar la conexión al final
    return result

In [228]:
data=fetch_data(var)

Conexión exitosa.


C:\Users\kings\AppData\Local\Temp\ipykernel_16924\4172916895.py:66: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [229]:
data

,hoja_tramite,tipo_hoja,DESCRIPCION,INDICATIVO_OFICIO,RAZON_SOCIAL,FECHA_RECEPCION,ASUNTO,AUDIT_REC,SIGLAS_ORIGEN,DEPENDENCIA_ORIGEN,...,derivado,finalizado,DIFERENCIA_DIAS,ESTADO,Hoja de trámite,Trabajador,parte_izquierda,Destino,Destino_vf,tip_hoja
0,00007711-2024,E,OFICIO MULTIPLE,0179-2024-NIT-0001998,MINISTERIO DE SALUD,Feb 01 2024 02:43PM,INVITACION A REUNION PARA PRESENTACION DE PLAN...,2024-02-02 08:09:01.577,GG,GERENCIA GENERAL,...,0,0,235,Pendiente,00007711-2024-E,"DIONNI EUGENIA, CAVERO LEBANO",GCPS,None,GCPS - GERENCIA CENTRAL DE PRESTACIONES DE SALUD,Externo
1,00097258-2024,I,SIN DOCUMENTO,00000989-2024-OFIN/ESSALUD-CONSUELO.MORAN,None,Sep 11 2024 01:57PM,MUY URGENTE - Informar las solicitudes de acce...,2024-09-17 09:39:09.170,GG,GERENCIA GENERAL,...,0,0,7,Pendiente,00097258-2024-I,"JORGE LUIS, MANRIQUE LLONTOP",OFIN,None,OFIN - OFICINA DE INTEGRIDAD,Interno
3,00097306-2024,I,SIN DOCUMENTO,00000998-2024-OFIN/ESSALUD-CONSUELO.MORAN,None,Sep 11 2024 02:55PM,MUY URGENTE - Informar las solicitudes de acce...,2024-09-13 18:26:26.863,GG,GERENCIA GENERAL,...,0,0,8,Pendiente,00097306-2024-I,"JORGE LUIS, MANRIQUE LLONTOP",OFIN,None,OFIN - OFICINA DE INTEGRIDAD,Interno
5,00046290-2024,E,NOTA,NOTA 616-GRPL-ESSALUD-2024,SEGURO SOCIAL DE SALUD,Jun 03 2024 08:34AM,NOTA 616-GRPL-ESSALUD-2024 NIT 1740-2024-49,2024-07-02 16:16:23.963,GG,GERENCIA GENERAL,...,0,0,84,Pendiente,00046290-2024-E,"CONSUELO INES, MORAN SAN MIGUEL",OFIN,None,OFIN - OFICINA DE INTEGRIDAD,Externo
9,00063837-2024,E,CARTA,s/n 30-07-2024,ARKINCAD S.A.C.,Jul 30 2024 01:31PM,solicitud de reunion para ser informado sobre...,2024-09-20 19:25:52.090,GG,GERENCIA GENERAL,...,0,0,1,Pendiente,00063837-2024-E,"CONSUELO INES, MORAN SAN MIGUEL",OFIN,None,OFIN - OFICINA DE INTEGRIDAD,Externo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1766,00043748-2024,I,PROYECTO DE RESPUESTA,00000232-2024-PE/ESSALUD-GIULIANA.DELCASTILLO,None,Apr 18 2024 07:08AM,Remisión de cronograma de actividades y adopci...,2024-04-19 17:47:12.537,PE,PRESIDENCIA EJECUTIVA,...,0,0,155,Pendiente,00043748-2024-I,"general02, gerencia",GG,None,GG - GERENCIA GENERAL,Interno
1771,00059685-2024,E,CARTA,S/N,None,Jul 12 2024 09:51AM,ACLARACION A NOTICIA DIARIO EXPRESO 12-07-2024,2024-07-19 12:50:17.807,GG,GERENCIA GENERAL,...,0,0,67,Pendiente,00059685-2024-E,"CONSUELO INES, MORAN SAN MIGUEL",OFIN,None,OFIN - OFICINA DE INTEGRIDAD,Externo
1775,00089103-2024,I,MEMORANDO,00000306-2024-PE/ESSALUD,None,Aug 21 2024 11:31AM,Conformación de Equipo de Trabajo para la elab...,2024-09-17 12:29:00.663,GG,GERENCIA GENERAL,...,0,0,7,Pendiente,00089103-2024-I,"KEILE JOYCE, DESPOSORIO ROBLES",PE,None,PE - PRESIDENCIA EJECUTIVA,Interno
1777,00000493-2024,E,SOLICITUD,,CENTRAL AUTONOMA DE TRABAJADORES DEL PERU - CA...,Jan 08 2024 10:25AM,SOLICITA INVESTIGACION NDE APROPIACION ILICITA...,2024-01-08 22:20:26.253,PE,PRESIDENCIA EJECUTIVA,...,0,0,258,Pendiente,00000493-2024-E,"YOSY, VERAMENDI REYNA",GG,None,GG - GERENCIA GENERAL,Externo


In [230]:
data.columns

Index(['hoja_tramite', 'tipo_hoja', 'DESCRIPCION', 'INDICATIVO_OFICIO',
       'RAZON_SOCIAL', 'FECHA_RECEPCION', 'ASUNTO', 'AUDIT_REC',
       'SIGLAS_ORIGEN', 'DEPENDENCIA_ORIGEN', 'SIGLAS_DESTINO',
       'DEPENDENCIA_DESTINO', 'APELLIDOS_TRABAJADOR', 'NOMBRES_TRABAJADOR',
       'ID_ESTADO_DOCUMENTO', 'ESTADO_DOCUMENTO', 'FECHA_DELEGADO', 'derivado',
       'finalizado', 'DIFERENCIA_DIAS', 'ESTADO', 'Hoja de trámite',
       'Trabajador', 'parte_izquierda', 'Destino', 'Destino_vf', 'tip_hoja'],
      dtype='object')

In [231]:
tipdoc= data["tip_hoja"].unique()
options_razon_social=data["RAZON_SOCIAL"].dropna().unique()
options_razon_social

array(['MINISTERIO DE SALUD', 'SEGURO SOCIAL DE SALUD', 'ARKINCAD S.A.C.',
       'PRESIDENCIA DEL CONSEJO DE MINISTROS', 'CLASSIS CORP. S.R.L.',
       'FONDO NACIONAL DE FINANCIAMIENTO DE LA A CTIVIDAD EMPRESARIAL DEL ESTADO - FONAFE O FONAFE',
       'CONTRALORIA GENERAL DE LA REPUBLICA', 'CONGRESO DE LA REPUBLICA',
       'MINIST.DE TRABAJO Y PROMOCION DEL EMPLEO',
       'OFICINA DE LAS NACIONES UNIDAS DE SERVICIOS PARA PROYECTOS - UNOPS',
       'MINISTERIO DE JUSTICIA Y DERECHOS HUMANOS',
       'ASOCIACION PARA EL FOMENTO DE LA INFRAESTRUCTURA NACIONAL - AFIN',
       'MINISTERIO DE ECONOMIA Y FINANZAS', 'ASTRAZENECA PERU S.A.',
       'CENTRAL AUTONOMA DE TRABAJADORES DEL PERU - CATP - PERU',
       'GERMAN CAMARENA DAVID VICENTE', 'SALOG S.A.',
       'CLINICA UNIVERSIDAD DE NAVARRA'], dtype=object)

In [232]:
def get_unique_options():
    connection = create_connection()
    if connection:
        try:
            cursor = connection.cursor()
            query = """SELECT DEPENDENCIA
                       FROM DB_GENERAL.jcardenas.H_DEPENDENCIA"""
            cursor.execute(query)
            # Retornar solo los valores de la columna 'DEPENDENCIA'
            options = [row[0] for row in cursor.fetchall()]
            cursor.close()
            return options
        finally:
            connection.close()
    return []

# Llamada a la función para obtener los valores de la columna 'DEPENDENCIA'
options = get_unique_options()

Conexión exitosa.


In [233]:
external_stylesheets = [
    dbc.themes.BOOTSTRAP,
    'https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css',  # Bootstrap CSS
    'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.1/css/all.min.css'  # Bootstrap Icons
]

# Crear la aplicación Dash
app = dash.Dash(__name__, suppress_callback_exceptions=True, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
    
    # Título
dbc.Row([
    # Título y logotipo en la misma fila
    dbc.Col([
        html.Div(style={'height': '12px'}),
        html.H1("Seguimiento de trámites pendientes", style={'color': '#0064AF', 'fontSize': '28px', 'textAlign': 'Left', 'font-weight': 'bold', 'fontFamily': 'Calibri'}),
        html.H2("Fuente: SGD Sede Central", style={'color': '#0064AF', 'fontSize': '12px'}),
    ], width=10),  # Ajusta el ancho según sea necesario
    dbc.Col([
        html.Img(src="/assets/Logotipo sin Slogan_Horizontal_Color Transparente.png", alt="Essalud", width="150"),
    ], width=2, className="d-none d-lg-flex align-items-center justify-content-end"),
]),
    html.Hr(style={'border': '2px solid #0064AF'}),
    dbc.Row([
        # Date Picker for Start Date
        dbc.Col([
            html.Label("Hoja de trámite", style={'font-size': '16px', 'color': '#0064AF'}),
            dcc.Input(id='HT',style={'font-size': '12px', 'height': '40px'}),
                ], width=9, md=3, lg=1),

        dbc.Col([
            html.Label("Tipo de documento", style={'font-size': '16px', 'color': '#0064AF'}),
            dcc.Dropdown(
                id='tipdoc',
                options=tipdoc,
                placeholder="Seleccione tipo",
                optionHeight=35,
                style={'height': '41px', 'width': '100%'}
        ),],width=9, md=3, lg=2),

        dbc.Col([
            html.Label("Razón social", style={'font-size': '16px', 'color': '#0064AF'}),
            dcc.Dropdown(
                id='razsoc',
                options=options_razon_social,
                placeholder="Seleccione R.S.",
                optionHeight=60,
                style={'height': '41px', 'width': '100%', 'font-size': '16px'}
        ),],width=9, md=3, lg=3),

        dbc.Col([
            html.Label("Fecha delegado", style={'font-size': '16px', 'color': '#0064AF'}),
            dcc.DatePickerRange(
                id='date-picker-range',
                minimum_nights=0,
                start_date_placeholder_text="Fecha inicio",
                end_date_placeholder_text="Fecha fin",
                display_format='DD-MM-YYYY',
                style={'height': '100px','font-size': '13px' },),
                
            html.Div(id='output-container')],width=9, md=3, lg=2),
            dbc.Col(
            
        ),

    ], className="mb-4"),

        # Salida de datos con dbc.Spinner
        dbc.Spinner(
            id="loading-output1",
            size="md",
            color="primary",
            type="border",
            fullscreen=False,
            children=html.Div(id='output-data-table-tab1')
        ),
        dbc.Alert(id="error-alert1", is_open=False, dismissable=True, color="danger"),

], fluid=True)



@app.callback(
    Output('output-data-table-tab1', 'children'),
    Output('error-alert1', 'is_open'),
    Output('error-alert1', 'children'), 
    Input('HT', 'value'),
    Input('tipdoc', 'value'),
    Input('razsoc', 'value'),
    Input('date-picker-range', 'start_date'),
    Input('date-picker-range', 'end_date')
)
def update_table(hoja_tramite, tipo_doc, razon_social, start_date, end_date):
    # Copiamos el DataFrame original
    df_filtered = data

    # Filtros
    if hoja_tramite:
        df_filtered = df_filtered[df_filtered['hoja_tramite'].str.contains(hoja_tramite, na=False, case=False)]    
    if tipo_doc:
        df_filtered = df_filtered[df_filtered['tip_hoja'] == tipo_doc]
    
    if razon_social:
        df_filtered = df_filtered[df_filtered['RAZON_SOCIAL'] == razon_social]
    
    if start_date and end_date:
        df_filtered = df_filtered[
            (df_filtered['FECHA_DELEGADO'] >= start_date) & 
            (df_filtered['FECHA_DELEGADO'] <= end_date)
        ]

    # Si no hay datos, muestra el alerta de error
    if df_filtered.empty:
        error_message = "No se encontraron registros"
        return html.Div(),True,error_message
    
    # Crea la tabla con el DataFrame filtrado
    table = dash_table.DataTable(
      id='data-table',
                    columns=[{"name": i, "id": i} for i in ["Hoja de trámite","DESCRIPCION","INDICATIVO_OFICIO","FECHA_DELEGADO","RAZON_SOCIAL","ASUNTO","DEPENDENCIA_ORIGEN","Destino_vf","Trabajador","DIFERENCIA_DIAS"]],
                    data=df_filtered.to_dict('records'),
                    style_table={'overflowX': 'auto', 'width': '100%',},
                    page_size=10,
                    style_cell={
                            'textAlign': 'left',
                            'fontFamily': 'Calibri',
                            'padding': '5px',
                            'height': 'auto',
                            'maxWidth': '120px',
                            'whiteSpace': 'normal',
                            'color': '#606060',
                            'fontSize': '14px'
                    },
                    style_header={
                        'backgroundColor': '#0064AF',
                        'fontWeight': 'bold',
                        'color': 'white'
                    },
                    style_cell_conditional=[
                        {'if': {'column_id': 'Hoja de trámite'}, 'minWidth': '100px', 'width': '100px', 'maxWidth': '200px', 'textAlign': 'center'},
                        {'if': {'column_id': 'DESCRIPCION'}, 'minWidth': '100px', 'width': '100px', 'maxWidth': '250px', 'textAlign': 'center'},
                        {'if': {'column_id': 'INDICATIVO_OFICIO'}, 'minWidth': '80px', 'width': '80px', 'maxWidth': '100px', 'textAlign': 'center'},
                        {'if': {'column_id': 'FECHA_DELEGADO'}, 'minWidth': '80px', 'width': '80px', 'maxWidth': '100px', 'textAlign': 'center'},
                        {'if': {'column_id': 'RAZON_SOCIAL'}, 'minWidth': '100px', 'width': '100px', 'maxWidth': '100px', 'textAlign': 'center'},
                        {'if': {'column_id': 'ASUNTO'}, 'minWidth': '200px', 'width': '200px', 'maxWidth': '250px', 'textAlign': 'center'},
                        {'if': {'column_id': 'DEPENDENCIA_ORIGEN'}, 'minWidth': '100px', 'width': '100px', 'maxWidth': '100px', 'textAlign': 'center'},
                        {'if': {'column_id': 'Destino_vf'}, 'minWidth': '100px', 'width': '100px', 'maxWidth': '100px', 'textAlign': 'center'},
                        {'if': {'column_id': 'Trabajador'}, 'minWidth': '100px', 'width': '100px', 'maxWidth': '100px', 'textAlign': 'center'},
                        {'if': {'column_id': 'DIFERENCIA_DIAS'}, 'minWidth': '60px', 'width': '60px', 'maxWidth': '65px', 'textAlign': 'center'},
                    ],
                    style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(244, 250, 253)'}],
    )

    return table, False, ""


if __name__ == '__main__':
    app.run_server(debug=True)
